<a href="https://colab.research.google.com/github/adnil8130/T5LittleGenQA/blob/main/T5%E7%94%9F%E6%88%90%E5%BC%8F%E9%97%AE%E7%AD%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 目标
训练一个生成式问答模型，base模型采用Google T5-Base("uer/t5-base-chinese-cluecorpussmall")

预训练模型地址：https://huggingface.co/uer/t5-base-chinese-cluecorpussmall

模型的评价指标采用BLEU-1，BLEU-2，BLEU-3，BLEU-4。

# 数据集
数据集：链接：https://pan.quark.cn/s/6d4a98cd65f2    

提取码：bzne

数据的格式如下：
```
{"context": "违规分为:一般违规扣分、严重违规扣分、出售假冒商品违规扣分,淘宝网每年12月31日24:00点会对符合条件的扣分做清零处理,详情如下:|温馨提醒:由于出售假冒商品24≤N<48分,当年的24分不清零,所以会存在第一年和第二年的不同计分情况。", "answer": "12月31日24:00", "question": "淘宝扣分什么时候清零", "id": 203}
```

# 1. 准备数据

In [37]:
# ! pip install transformers

In [38]:
import transformers
import numpy as np
import torch
print(transformers.__version__)
print(torch.__version__)
from torch.utils.data import Dataset, random_split
import json

4.48.3
2.5.1+cu124


In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
max_context_len = 0
max_question_len = 0
max_answer_len = 0
context = ""
questiont = ""
answer = ""

save_path_file = '/content/drive/MyDrive'
train_data_address = save_path_file + '/train.json'
dev_data_address = save_path_file + '/dev.json'

with open(train_data_address, 'rt', encoding='utf-8') as f:
  for idx, line in enumerate(f):
    sample = json.loads(line.strip())
    if len(sample["context"]) > max_question_len:
        max_context_len = len(sample["context"])
        context = sample["context"]
    if len(sample["question"]) > max_question_len:
        max_question_len = len(sample["question"])
        question = sample["question"]
    if len(sample["answer"]) > max_answer_len:
        max_answer_len = len(sample["answer"])
        answer = sample["answer"]

with open(dev_data_address, 'rt', encoding='utf-8') as f:
  for idx, line in enumerate(f):
    sample = json.loads(line.strip())
    if len(sample["context"]) > max_question_len:
        max_context_len = len(sample["context"])
        context = sample["context"]
    if len(sample["question"]) > max_question_len:
        max_question_len = len(sample["question"])
        question = sample["question"]
    if len(sample["answer"]) > max_answer_len:
        max_answer_len = len(sample["answer"])
        answer = sample["answer"]

print("最长context", max_context_len, context)
print("最长question", max_question_len, question)
print("最长answer", max_answer_len, answer)

最长context 107 2015下半年教师资格证考试时间为11月1日，考生可于2015年10月26日—10月31日登录报名系统，根据提示下载pdf准考证文件。下载后，仔细核对个人信息，并直接打印成准考，按准考证上的要求到指定地点参加考试。
最长question 42 痞子猪身上是什么字母? (问题由猫小逗提供)【答题格式为da+答案,例如答案是爱消除
最长answer 110 如果下雨的时候你拖着行李箱子站在屋檐下面那么其实我没有足够的时间找一个好一点的理由抛弃家里面的狗坐上K667次列车到你在的地方找个商店买一把伞然后给我妹妹弹吉他因为她要参加比赛所以我回不去了我也不会给你说我泡面的碗还没洗


## 构建数据集

In [41]:
train_ratio = 0.9
max_dataset_size = 22000
train_set_size = 20000
valid_set_size = 2000

class GenQA(Dataset):
  def __init__(self, data_file):
    self.data = self.load_data(data_file)

  def load_data(self, data_file):
    Data = {}
    with open(data_file, 'rt', encoding='utf-8') as f:
      for idx, line in enumerate(f):
        if idx >= max_dataset_size:
            break
        sample = json.loads(line.strip())
        Data[idx] = sample
    return Data

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    return self.data[idx]

data = GenQA(train_data_address)
data_size = len(data)

train_size = int(train_ratio * data_size)
valid_size = data_size - train_size
train_data, valid_data = random_split(data, [train_size, valid_size])
test_data = GenQA(dev_data_address)

In [42]:
print(f'train set size: {len(train_data)}')
print(f'valid set size: {len(valid_data)}')
print(f'test set size: {len(test_data)}')
print(next(iter(train_data)))

train set size: 13068
valid set size: 1452
test set size: 984
{'context': 'JPT HOME是以高性价比为基础的快时尚买手品牌，主要经营和销售服装、鞋履、配饰和日常生活必备品，集研发、设计、供应、销售于一体，专注于提供优质、平价、时尚的男装快消产品。|JPT HOME自创立以来，始终坚持自已的设计路线，追求时尚、简约而舒适的设计风格，在男装快时尚领域一展风华。', 'answer': '快时尚买手品牌', 'question': 'jpthome是什么牌子', 'id': 6847}


## 数据预处理

In [43]:
from transformers import AutoTokenizer
model_checkpoint = 'uer/t5-base-chinese-cluecorpussmall'
# model_checkpoint = 'uer/t5-small-chinese-cluecorpussmall'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,return_token_type_ids=False)

In [44]:
context = train_data[0]["context"]
question = train_data[0]["question"]
answer = train_data[0]["answer"]

inputs = tokenizer(context, question)
targets = tokenizer(answer)

In [45]:
print(tokenizer.convert_ids_to_tokens(inputs["input_ids"]))
print(tokenizer.convert_ids_to_tokens(targets["input_ids"]))

['[CLS]', '[UNK]', '[UNK]', '是', '以', '高', '性', '价', '比', '为', '基', '础', '的', '快', '时', '尚', '买', '手', '品', '牌', '，', '主', '要', '经', '营', '和', '销', '售', '服', '装', '、', '鞋', '履', '、', '配', '饰', '和', '日', '常', '生', '活', '必', '备', '品', '，', '集', '研', '发', '、', '设', '计', '、', '供', '应', '、', '销', '售', '于', '一', '体', '，', '专', '注', '于', '提', '供', '优', '质', '、', '平', '价', '、', '时', '尚', '的', '男', '装', '快', '消', '产', '品', '。', '|', '[UNK]', '[UNK]', '自', '创', '立', '以', '来', '，', '始', '终', '坚', '持', '自', '已', '的', '设', '计', '路', '线', '，', '追', '求', '时', '尚', '、', '简', '约', '而', '舒', '适', '的', '设', '计', '风', '格', '，', '在', '男', '装', '快', '时', '尚', '领', '域', '一', '展', '风', '华', '。', '[SEP]', 'jp', '##th', '##ome', '是', '什', '么', '牌', '子', '[SEP]']
['[CLS]', '快', '时', '尚', '买', '手', '品', '牌', '[SEP]']


In [46]:
import torch

max_input_length = 256
max_target_length = 128
sample_cnt = 2

inputs = [train_data[s_idx]["context"] + train_data[s_idx]["question"] + "?" for s_idx in range(sample_cnt)]
targets = [train_data[s_idx]["answer"] for s_idx in range(sample_cnt)]

model_inputs = tokenizer(
    inputs,
    padding=True,
    max_length=max_input_length,
    truncation=True,
    return_tensors="pt",
    return_token_type_ids=False
)
labels = tokenizer(
    text_target=targets,
    padding=True,
    max_length=max_target_length,
    truncation=True,
    return_tensors="pt",
    return_token_type_ids=False
)["input_ids"]

end_token_index = torch.where(labels == 102)[-1]
for idx, end_idx in enumerate(end_token_index):
    labels[idx][end_idx+1:] = -100

print('batch_X shape:', {k: v.shape for k, v in model_inputs.items()})
print('batch_y shape:', labels.shape)
print(model_inputs)
print(labels)

batch_X shape: {'input_ids': torch.Size([2, 256]), 'attention_mask': torch.Size([2, 256])}
batch_y shape: torch.Size([2, 9])
{'input_ids': tensor([[  101,   100,   100,  3221,   809,  7770,  2595,   817,  3683,   711,
          1825,  4794,  4638,  2571,  3198,  2213,   743,  2797,  1501,  4277,
          8024,   712,  6206,  5307,  5852,  1469,  7218,  1545,  3302,  6163,
           510,  7490,  2252,   510,  6981,  7652,  1469,  3189,  2382,  4495,
          3833,  2553,  1906,  1501,  8024,  7415,  4777,  1355,   510,  6392,
          6369,   510,   897,  2418,   510,  7218,  1545,   754,   671,   860,
          8024,   683,  3800,   754,  2990,   897,   831,  6574,   510,  2398,
           817,   510,  3198,  2213,  4638,  4511,  6163,  2571,  3867,   772,
          1501,   511,   170,   100,   100,  5632,  1158,  4989,   809,  3341,
          8024,  1993,  5303,  1780,  2898,  5632,  2347,  4638,  6392,  6369,
          6662,  5296,  8024,  6841,  3724,  3198,  2213,   510,  5042,

In [47]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoModelForSeq2SeqLM
import gc

train_batch_size = 64
test_batch_size = 64

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

def clean_cuda(device):
    if device == 'cuda':
        # 清理无用变量
        gc.collect()
        torch.cuda.empty_cache()

clean_cuda(device)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
model = model.to(device)

def collote_fn(batch_samples):
    batch_inputs, batch_targets = [], []
    for sample in batch_samples:
        batch_inputs.append(sample["context"] + sample["question"] + "?")
        batch_targets.append(sample['answer'])
    batch_data = tokenizer(
        batch_inputs,
        text_target=batch_targets,
        padding=True,
        max_length=max_input_length,
        truncation=True,
        return_tensors="pt",
        return_token_type_ids=False
    )

    # with tokenizer.as_target_tokenizer():
    #     labels = tokenizer(
    #         batch_targets,
    #         padding=True,
    #         max_length=max_target_length,
    #         truncation=True,
    #         return_tensors="pt"
    #     )["input_ids"]

    batch_data['decoder_input_ids'] = model.prepare_decoder_input_ids_from_labels(batch_data['labels'])
    # end_token_index = torch.where(batch_data['labels'] == 102)[-1]
    # for idx, end_idx in enumerate(end_token_index):
    #     batch_data['labels'][idx][end_idx+1:] = -100
    return batch_data

train_dataloader = DataLoader(train_data, batch_size=train_batch_size, shuffle=True, collate_fn=collote_fn)
valid_dataloader = DataLoader(valid_data, batch_size=test_batch_size, shuffle=False, collate_fn=collote_fn)

Using cuda device


In [48]:
batch = next(iter(train_dataloader))
print(batch.keys())
print('batch shape:', {k: v.shape for k, v in batch.items()})
print(batch)

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])
batch shape: {'input_ids': torch.Size([64, 256]), 'attention_mask': torch.Size([64, 256]), 'labels': torch.Size([64, 18]), 'decoder_input_ids': torch.Size([64, 18])}
{'input_ids': tensor([[ 101, 1101, 2207,  ...,    0,    0,    0],
        [ 101, 6612, 2209,  ..., 1044, 2797,  102],
        [ 101, 2418, 6421,  ...,    0,    0,    0],
        ...,
        [ 101, 1872,  966,  ...,    0,    0,    0],
        [ 101,  517, 3905,  ..., 3221, 2042,  102],
        [ 101,  782, 1366,  ...,    0,    0,    0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[  101,  1101,  2207,  ...,     0,     0,     0],
        [  101,  9866,  4294,  ...,     0,     0,     0],
        [  101,  8119,  2399,  ...,     0,     0,     0]

# 2. 模型训练

## 优化模型参数

In [49]:
from tqdm.auto import tqdm
import random

def train_loop(dataloader, model, optimizer, lr_scheduler, epoch, total_loss):
    progress_bar = tqdm(range(len(dataloader)))
    progress_bar.set_description(f'loss: {0:>7f}')
    finish_batch_num = (epoch-1) * len(dataloader)

    model.train()
    loss_record_step = []
    for batch, batch_data in enumerate(dataloader, start=1):
        batch_data = batch_data.to(device)
        outputs = model(**batch_data)

        random_number = random.uniform(0, 1)

        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        total_loss += loss.item()
        loss_record_step.append(loss.item())
        progress_bar.set_description(f'loss: {total_loss/(finish_batch_num + batch):>7f}')
        progress_bar.update(1)
        del batch_data, outputs, loss
        clean_cuda(device)
    return total_loss, loss_record_step

## 评估指标

In [50]:
# ! pip install sacrebleu

In [51]:
from sacrebleu.metrics import BLEU

predictions = [
    "This plugin lets you translate web pages between several languages automatically."
]
bad_predictions_1 = ["This This This This"]
bad_predictions_2 = ["This plugin"]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]

bleu1 = BLEU(max_ngram_order=1)
print("==========BLEU-1==========")
print(bleu1.corpus_score(predictions, references).score)
print(bleu1.corpus_score(bad_predictions_1, references).score)
print(bleu1.corpus_score(bad_predictions_2, references).score)

bleu2 = BLEU(max_ngram_order=2)
print("==========BLEU-2==========")
print(bleu2.corpus_score(predictions, references).score)
print(bleu2.corpus_score(bad_predictions_1, references).score)
print(bleu2.corpus_score(bad_predictions_2, references).score)

bleu3 = BLEU(max_ngram_order=3)
print("==========BLEU-3==========")
print(bleu3.corpus_score(predictions, references).score)
print(bleu3.corpus_score(bad_predictions_1, references).score)
print(bleu3.corpus_score(bad_predictions_2, references).score)

bleu4 = BLEU(max_ngram_order=4)
print("==========BLEU-4==========")
print(bleu4.corpus_score(predictions, references).score)
print(bleu4.corpus_score(bad_predictions_1, references).score)
print(bleu4.corpus_score(bad_predictions_2, references).score)

==========BLEU-1==========
84.33740467435464
2.634980614046608
0.40867714384640685
==========BLEU-2==========
65.05696445772017
2.1514526621798953
0.40867714384640685
==========BLEU-3==========
53.804523766396244
1.8269935164445736
0.0
==========BLEU-4==========
46.750469682990165
1.683602693167689
0.0


In [52]:
import random

def test_loop(dataloader, model, sample_ovserve_ratio=0.05):
    preds, labels = [], []

    model.eval()
    for batch_data in tqdm(dataloader):
        batch_data = batch_data.to(device)
        with torch.no_grad():
            generated_tokens = model.generate(
                batch_data["input_ids"],
                attention_mask=batch_data["attention_mask"],
                max_length=max_target_length,
            ).cpu().numpy()
        label_tokens = batch_data["labels"].cpu().numpy()

        decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

        random_number = random.uniform(0, 1)
        if random_number < sample_ovserve_ratio:
            print("input:", tokenizer.batch_decode(batch_data["input_ids"].cpu().numpy(), skip_special_tokens=True))
            print("output:", decoded_preds)

        label_tokens = np.where(label_tokens != -100, label_tokens, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(label_tokens, skip_special_tokens=True)

        del batch_data
        clean_cuda(device)

        preds += [pred.strip() for pred in decoded_preds]
        labels += [[label.strip()] for label in decoded_labels]
    return bleu1.corpus_score(preds, labels).score, bleu2.corpus_score(preds, labels).score, bleu3.corpus_score(preds, labels).score, bleu4.corpus_score(preds, labels).score

## 保存模型

In [53]:
# test_data = GenQA(dev_data_address)
# test_dataloader = DataLoader(test_data, batch_size=test_batch_size, shuffle=False, collate_fn=collote_fn)

# test_loop(test_dataloader, model)

  0%|          | 0/16 [00:00<?, ?it/s]

(0.0, 0.0, 0.0, 0.0)

In [54]:
import matplotlib.pyplot as plt
import numpy as np

def save_data_and_plot(data, txt_file_path, img_file_path):
    # ===================== 写入文件部分 =====================
    # 追加写入数据（自动创建文件）
    with open(txt_file_path, 'a') as f:
        # 将数字转为字符串并换行写入
        f.write('\n'.join(map(str, data)))
        f.write('\n')  # 添加换行符分隔不同写入批次

    # ===================== 读取文件部分 =====================
    # 从文件读取所有数字
    loaded_data = []
    try:
        with open(txt_file_path, 'r') as f:
            for line in f:
                # 去除空白字符并尝试转换为浮点数
                cleaned_line = line.strip()
                if cleaned_line:
                    loaded_data.append(float(cleaned_line))
    except FileNotFoundError:
        print("错误：文件不存在")
        exit()

    # ===================== 绘图部分 =====================
    plt.figure(figsize=(10, 6))
    plt.plot(loaded_data,
            color='green',
            linestyle='--',
            marker='s',
            markersize=8,
            linewidth=2)

    # 图表装饰
    plt.title("数值变化曲线", fontsize=14, pad=20)
    plt.xlabel("数据索引", fontsize=12, labelpad=10)
    plt.ylabel("测量值", fontsize=12, labelpad=10)
    plt.grid(True, alpha=0.4, linestyle=':')

    # 自动调整坐标轴范围
    plt.xlim(0, len(loaded_data)-1)
    plt.ylim(min(loaded_data)-1, max(loaded_data)+1)

    # 保存和显示
    plt.savefig(img_file_path, dpi=300, bbox_inches='tight')
    plt.close()  # 关闭图表释放内存

    print("操作结果：")
    print(f"- 数据已保存至 {txt_file_path}")
    print(f"- 生成曲线图：{img_file_path}")
    print(f"- 加载数据量：{len(loaded_data)} 条")

In [57]:
from transformers import AdamW, get_scheduler

learning_rate = 2e-5
epoch_num = 61

optimizer = AdamW(model.parameters(), lr=learning_rate)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=epoch_num*len(train_dataloader),
)

total_loss = 0.
best_bleu1 = 0.
best_bleu2 = 0.
best_bleu3 = 0.
best_bleu4 = 0.
best_bleu_weighted_add = 0.
txt_file_path = save_path_file + '/lossdata.txt'
img_file_path = save_path_file + '/lossdata.png'
model_parm_path = None

train_batch_size = 32
test_batch_size = 32

train_dataloader = DataLoader(train_data, batch_size=train_batch_size, shuffle=True, collate_fn=collote_fn)
valid_dataloader = DataLoader(valid_data, batch_size=test_batch_size, shuffle=False, collate_fn=collote_fn)

if model_parm_path is not None:
    model.load_state_dict(torch.load(model_parm_path))

for t in range(epoch_num):
    clean_cuda(device)
    print(f"Epoch {t+1}/{epoch_num}\n-------------------------------")

    # 查看清理后剩余显存
    print(f"可用显存: {torch.cuda.mem_get_info()[0]/1024**3:.2f} GB")

    total_loss, loss_record_step = train_loop(train_dataloader, model, optimizer, lr_scheduler, t+1, total_loss)
    save_data_and_plot(loss_record_step, txt_file_path, img_file_path)

    # 查看清理后剩余显存
    print(f"可用显存: {torch.cuda.mem_get_info()[0]/1024**3:.2f} GB")
    valid_bleu1, valid_bleu2, valid_bleu3, valid_bleu4 = test_loop(valid_dataloader, model)
    print(f"BLEU1: {valid_bleu1:>0.2f}\n")
    if valid_bleu1 > best_bleu1:
        best_bleu1 = valid_bleu1
    print(f"BLEU2: {valid_bleu2:>0.2f}\n")
    if valid_bleu2 > best_bleu2:
        best_bleu2 = valid_bleu2
    print(f"BLEU3: {valid_bleu3:>0.2f}\n")
    if valid_bleu3 > best_bleu3:
        best_bleu3 = valid_bleu3
    print(f"BLEU4: {valid_bleu4:>0.2f}\n")
    if valid_bleu4 > best_bleu4:
        best_bleu4 = valid_bleu4

    valid_bleu = 0.1 * valid_bleu1 + 0.2 * valid_bleu2 + 0.3 * valid_bleu3 + 0.4 * valid_bleu4
    if valid_bleu > best_bleu_weighted_add or t % 5 == 0:
        best_bleu_weighted_add = valid_bleu
        print('saving new weights...\n')
        torch.save(model.state_dict(), save_path_file + f'/epoch_{t+1}_loss_{loss_record_step[-1]:0.7f}_valid_bleu_{valid_bleu:0.2f}_model_weights.bin')
print("Done!")

Epoch 1/61
-------------------------------
可用显存: 11.38 GB


  0%|          | 0/409 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 18.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 12.12 MiB is free. Process 4530 has 14.73 GiB memory in use. Of the allocated memory 14.15 GiB is allocated by PyTorch, and 457.14 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
test_data = GenQA(dev_data_address)
test_dataloader = DataLoader(test_data, batch_size=test_batch_size, shuffle=False, collate_fn=collote_fn)

import json

model.load_state_dict(torch.load('./epoch_56_loss_4.8707561_valid_bleu_0.00_model_weights (1).bin'))

model.eval()
with torch.no_grad():
    print('evaluating on test set...')
    sources, preds, labels = [], [], []
    for batch_data in tqdm(test_dataloader):
        batch_data = batch_data.to(device)
        generated_tokens = model.generate(
            batch_data["input_ids"],
            attention_mask=batch_data["attention_mask"],
            max_length=max_target_length,
        ).cpu().numpy()
        label_tokens = batch_data["labels"].cpu().numpy()

        decoded_sources = tokenizer.batch_decode(
            batch_data["input_ids"].cpu().numpy(),
            skip_special_tokens=True,
            use_source_tokenizer=True
        )
        # print('decoded_sources', decoded_sources)

        decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        # print('decoded_preds', decoded_preds)
        label_tokens = np.where(label_tokens != -100, label_tokens, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(label_tokens, skip_special_tokens=True)
        # print('decoded_labels', decoded_labels)

        sources += [source.strip() for source in decoded_sources]
        preds += [pred.strip() for pred in decoded_preds]
        labels += [[label.strip()] for label in decoded_labels]
    bleu_score = bleu4.corpus_score(preds, labels).score
    print(f"Test BLEU: {bleu_score:>0.2f}\n")
    results = []
    print('saving predicted results...')
    for source, pred, label in zip(sources, preds, labels):
        results.append({
            "sentence": source,
            "prediction": pred,
            "translation": label[0]
        })
    with open('test_data_pred.json', 'wt', encoding='utf-8') as f:
        for exapmle_result in results:
            f.write(json.dumps(exapmle_result, ensure_ascii=False) + '\n')

In [ ]:
model.load_state_dict(torch.load('./epoch_61_loss_4.9312925_valid_bleu_0.00_model_weights.bin'))
test_dataloader1 = DataLoader(test_data, batch_size=1, shuffle=False, collate_fn=collote_fn)
test_loop(test_dataloader1, model, sample_ovserve_ratio=1)


/tmp/ipykernel_1149/442049437.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('./epoch_61_loss_4.9312925_valid_bleu_0.00_model_weights.b

  0%|          | 0/984 [00:00<?, ?it/s]

input: ['年 基 准 利 率 4. 35 % 。 从 实 际 看, 贷 款 的 基 本 条 件 是 : 一 是 中 国 大 陆 居 民, 年 龄 在 60 岁 以 下 ; 二 是 有 稳 定 的 住 址 和 工 作 或 经 营 地 点 ; 三 是 有 稳 定 的 收 入 来 源 ; 四 是 无 不 良 信 用 记 录, 贷 款 用 途 不 能 作 为 炒 股, 赌 博 等 行 为 ; 五 是 具 有 完 全 民 事 行 为 能 力 。 2017 年 银 行 贷 款 基 准 利 率']
output: ['']
input: ['年 基 准 利 率 4. 35 % 。 从 实 际 看, 贷 款 的 基 本 条 件 是 : 一 是 中 国 大 陆 居 民, 年 龄 在 60 岁 以 下 ; 二 是 有 稳 定 的 住 址 和 工 作 或 经 营 地 点 ; 三 是 有 稳 定 的 收 入 来 源 ; 四 是 无 不 良 信 用 记 录, 贷 款 用 途 不 能 作 为 炒 股, 赌 博 等 行 为 ; 五 是 具 有 完 全 民 事 行 为 能 力 。 2017 年 银 行 贷 款 基 准 利 率']
output: ['']


KeyboardInterrupt: 